In [111]:
import pandas as pd
import numpy as np
  
from string import punctuation
full_punctuation = punctuation + "–" + "," + "»" + "«" + "…" +'’'

In [86]:
from gensim.models.wrappers import FastText
model_fr = FastText.load_fasttext_format('/Users/nigula/input/fr/fr.bin')

In [47]:
from gensim.models.keyedvectors import FastTextKeyedVectors


In [76]:
from  sklearn.metrics.pairwise import cosine_similarity


In [48]:
model_rus = FastTextKeyedVectors.load('/Users/nigula/input/araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model')


# one_element evaluation

In [10]:
reference_dict = pd.read_csv("/Users/nigula/LL/adjust_unigr_dict/lookup results/reference_dict.csv")
reference_dict.head()

,English,Russian,French
0,letter,письмо,lettre
1,letter,буква,lettre
2,letter,буква,caractère
3,plate,тарелка,assiette
4,plate,тарелка,plat


In [52]:
def update_dict (ref_dict_eng_word, l1_w, l2_w):
    if l1_w in ref_dict_eng_word:
        ref_dict_eng_word[l1_w].append((l1_w, l2_w))
    else:
        ref_dict_eng_word[l1_w] = []
        ref_dict_eng_word[l1_w].append((l1_w, l2_w))

In [53]:
def reference_to_dict (df):
    ref_dict = {}
    for df_ind, val in df.iterrows():
        #print(list(val))
        
        val_list = list(val)
        eng_word = val_list[0]
        l1_word = val_list[1] 
        l2_word = val_list[2]
        if eng_word in ref_dict:
            update_dict(ref_dict[eng_word],l1_word, l2_word)
            #ref_dict[eng_word].append((l1_word, l2_word))
        else:
            ref_dict[eng_word] = {}
            update_dict(ref_dict[eng_word],l1_word, l2_word)
            #ref_dict[eng_word].append((l1_word, l2_word))
        #break
    return ref_dict
    
ref_dct = reference_to_dict(reference_dict)

In [54]:
ref_dct['break']

{'пауза': [('пауза', 'pause ')],
 'перерыв': [('перерыв', 'vacances')],
 'поломка, скол': [('поломка, скол', 'félure')],
 'ломать': [('ломать', 'briser'), ('ломать', 'casser')],
 'игнорировать': [('игнорировать', 'désobéir à')],
 'пренебречь': [('пренебречь', 'manquer à')],
 'прерывать': [('прерывать', 'interrompre')],
 'разразиться': [('разразиться', 'éclater')]}

In [35]:
merge_cont = pd.read_csv("/Users/nigula/LL/adjust_unigr_dict/reverse_connected_dicts/systran_merge_mean_04_similarity_vs_pos.csv")

In [36]:
merge_cont.head()

,Unnamed: 0,eng_word,l1,l1_context,l2,l2_context
0,0,treat,gâterie,wholesome treat|sweet treats|,угощение,morning treat|feast |
1,1,treat,gâterie,wholesome treat|sweet treats|,удовольствие,pleasure | fun | satisfaction | enjoyment | am...
2,2,treat,soigner,to treat people|to treat angina|cure | heal | ...,лечить,to treat people|heal | cure | heal | dose |
3,3,treat,soigner,to treat people|to treat angina|cure | heal | ...,вылечить,cure | heal |
4,4,treat,considérer,treated on a case-by-case basis|treated confid...,рассматривать,consider | see | view | regard | examine | rev...


In [38]:
merge_cont_short = pd.DataFrame()
merge_cont_short['en'] = merge_cont['eng_word']
merge_cont_short['rus'] = merge_cont['l2']
merge_cont_short['fr'] = merge_cont['l1']
merge_cont_short.head()


,en,rus,fr
0,treat,угощение,gâterie
1,treat,удовольствие,gâterie
2,treat,лечить,soigner
3,treat,вылечить,soigner
4,treat,рассматривать,considérer


In [55]:
systran_merge_mean_04_similarity_vs_pos_dict = reference_to_dict(merge_cont_short)

In [128]:
systran_merge_mean_04_similarity_vs_pos_dict['star']

{'звезда': [('звезда', 'étoile'),
  ('звезда', 'star'),
  ('звезда', 'vedette'),
  ('звезда', 'étoile'),
  ('звезда', 'star'),
  ('звезда', 'vedette')],
 'стар': [('стар', 'étoile'), ('стар', 'étoile')]}

In [129]:
ref_dct['star']

{'звезда': [('звезда', 'étoile'), ('звезда', 'vedette'), ('звезда', 'star ')],
 'звезда (первый ученик)': [('звезда (первый ученик)', 'meilleur')],
 'играть главную роль': [('играть главную роль', 'être la vedette'),
  ('играть главную роль', 'tenir le rôle principal')]}

In [73]:
a = ('пауза', 'no_equality')
'no_equality' in a

True

In [116]:
#l1 = "maman"
#l2 = 'mère de famille'
l1 = 'поломка, скол'
l2 = 'сломать'
 

def get_vectors_sum(lemm_line, model):
    orig_vector = np.zeros(shape = 300)
    for lemma in lemm_line.split():
        #print(lemma)
        try:
            orig_vector += model[lemma]
        except:
            pass
    return orig_vector

def find_lines_similarity(line_1, line_2, model):
    line_1_clear = ''
    for line_1_char in line_1:
        if line_1_char not in full_punctuation:
            line_1_clear += line_1_char
    line_2_clear = ''
    for line_2_char in line_2:
        if line_2_char not in full_punctuation:
            line_2_clear += line_2_char
    l1_vect = get_vectors_sum(line_1_clear, model)
    l2_vect = get_vectors_sum(line_2_clear, model)
    cos_sim = cosine_similarity(l1_vect.reshape(1, -1), l2_vect.reshape(1, -1))[0][0]
    #print(cos_sim)
    return cos_sim

find_lines_similarity(l1, l2, model_rus)    
    

0.3162484806225587

In [135]:
def eval_pair_similarity(pair_1, pair_2):
    if 'no_equality' in pair_1 or 'no_equality' in pair_2:
        return False
    else:
        l1_word_p1 = pair_1[0]
        l1_word_p2 = pair_2[0]
        #print(l1_word_p1, l1_word_p2)
        
        l2_word_p1 =  pair_1[1]
        l2_word_p2 = pair_2[1]
        #print(l2_word_p1, l2_word_p2)
        l1_words_sim = find_lines_similarity(l1_word_p1, l1_word_p2, model_rus)
        if l1_words_sim > 0.6:
            l2_words_sim = find_lines_similarity(l2_word_p1, l2_word_p2, model_fr)
            if l2_words_sim > 0.8:
                return True
    return "pisolse"
            
#eval_pair_similarity(('пауза', 'pause '),('пауза', 'no_equality') )
eval_pair_similarity(('перерыв', 'vacances'), ('перерыв', 'pause'))

'pisolse'

In [147]:
def evaluate_eng_word_quality(ref_dict_eng_word, ev_dict_eng_word, debug = False):
    all_var_reference_pairs = []
    for l1_word in ref_dict_eng_word.keys():
        for ref_pair in ref_dict_eng_word[l1_word]:
            ref_pair_line = ref_pair[0] + '_' + ref_pair[1]
            all_var_reference_pairs.append(ref_pair_line)
    all_var_reference_pairs_set = set(all_var_reference_pairs)
    all_var_ev_dict_pairs = []
    for l1_word_ev in ev_dict_eng_word.keys():
        for ev_pair in ev_dict_eng_word[l1_word_ev]:
            eval_pair_line = ev_pair[0] + "_" + ev_pair[1]
            all_var_ev_dict_pairs.append(eval_pair_line)
    all_var_ev_dict_pairs_set = set(all_var_ev_dict_pairs)
    
    handled_reference_pairs = []
    handled_ev_dict_pairs = []
    #get direct l1 word intersections
    for l1_word in ref_dict_eng_word.keys():
        if l1_word in ev_dict_eng_word:
            if debug == True: print(ref_dict_eng_word[l1_word], ev_dict_eng_word[l1_word])
            for ref_pair in ref_dict_eng_word[l1_word]:
                if debug == True: print(ref_pair)
                for eval_pair in ev_dict_eng_word[l1_word]:
                    if debug == True:print(ref_pair, eval_pair)
                    pair_sim_res = eval_pair_similarity(ref_pair, eval_pair)
                    if pair_sim_res == True:
                        if debug == True:print("positive_abs_eq")
                        ref_pair_line = ref_pair[0] + '_' + ref_pair[1]
                        handled_reference_pairs.append(ref_pair_line)
                        eval_pair_line = eval_pair[0] + "_" + eval_pair[1]
                        handled_ev_dict_pairs.append(eval_pair_line)
                    else:
                        if debug == True: print("NO_ABS_EQ")
                        pass
    print("=======DIRECT SIM FINISHED========") 
    print(handled_reference_pairs, handled_ev_dict_pairs)
    #get w2v similar l1 words_intersect
    eval_l1_words_list = list(ev_dict_eng_word.keys())
    for l1_word_ref in ref_dict_eng_word.keys():
        if l1_word_ref not in ev_dict_eng_word:
            for l1_word_eval in eval_l1_words_list:
                
                l1_word_sim = find_lines_similarity(l1_word_ref,l1_word_eval, model_rus)
                print(l1_word_ref,l1_word_eval,l1_word_sim)
                if l1_word_sim > 0.5:
                    print("l1_word_sim worked")
                    for ref_pair in ref_dict_eng_word[l1_word_ref]:
                        for eval_pair in ev_dict_eng_word[l1_word_eval]:
                            print(ref_pair, eval_pair)
                            pair_sim_res = eval_pair_similarity(ref_pair, eval_pair)
                            if pair_sim_res == True:
                                print("match_found")
                                ref_pair_line = ref_pair[0] + '_' + ref_pair[1]
                                handled_reference_pairs.append(ref_pair_line)
                                eval_pair_line = eval_pair[0] + "_" + eval_pair[1]
                                handled_ev_dict_pairs.append(eval_pair_line)
                            else:
                                print("no_match")
    print(handled_reference_pairs, handled_ev_dict_pairs)
    handled_reference_pairs_set = set(handled_reference_pairs)
    handled_ev_dict_pairs_set = set(handled_ev_dict_pairs)
    
    reference_intersect = handled_reference_pairs_set.intersection(all_var_reference_pairs_set)
    print(reference_intersect, len(reference_intersect)/len(all_var_reference_pairs_set))
    reference_dict_coverage = len(reference_intersect)/len(all_var_reference_pairs_set)
    
    eval_intersect = handled_ev_dict_pairs_set.intersection(all_var_ev_dict_pairs_set)
    print(eval_intersect, 1 - len(eval_intersect)/len(all_var_ev_dict_pairs_set), )
    non_paired_eval_pairs = 1 - len(eval_intersect)/len(all_var_ev_dict_pairs_set)
    
    return reference_dict_coverage, non_paired_eval_pairs
    
evaluate_eng_word_quality(ref_dct['star'], systran_merge_mean_04_similarity_vs_pos_dict['star'])       

=======DIRECT SIM FINISHED========
['звезда_étoile', 'звезда_étoile', 'звезда_vedette', 'звезда_vedette', 'звезда_star ', 'звезда_star '] ['звезда_étoile', 'звезда_étoile', 'звезда_vedette', 'звезда_vedette', 'звезда_star', 'звезда_star']
звезда (первый ученик) звезда 0.6145184998958503
l1_word_sim worked
('звезда (первый ученик)', 'meilleur') ('звезда', 'étoile')
no_match
('звезда (первый ученик)', 'meilleur') ('звезда', 'star')
no_match
('звезда (первый ученик)', 'meilleur') ('звезда', 'vedette')
no_match
('звезда (первый ученик)', 'meilleur') ('звезда', 'étoile')
no_match
('звезда (первый ученик)', 'meilleur') ('звезда', 'star')
no_match
('звезда (первый ученик)', 'meilleur') ('звезда', 'vedette')
no_match
звезда (первый ученик) стар 0.2641785284133709
играть главную роль звезда 0.060604119962037885
играть главную роль стар -0.03183579444872556
['звезда_étoile', 'звезда_étoile', 'звезда_vedette', 'звезда_vedette', 'звезда_star ', 'звезда_star '] ['звезда_étoile', 'звезда_étoile', 'з

(0.5, 0.25)

In [145]:
systran_merge_mean_04_similarity_vs_pos_dict['star']

{'звезда': [('звезда', 'étoile'),
  ('звезда', 'star'),
  ('звезда', 'vedette'),
  ('звезда', 'étoile'),
  ('звезда', 'star'),
  ('звезда', 'vedette')],
 'стар': [('стар', 'étoile'), ('стар', 'étoile')]}

In [155]:
def eval_dicts(ref_dict, eval_dict):
    eng_word_list = []
    ref_dict_coverage = []
    exceed_pairs_generation = []
    for ref_eng_word in ref_dict.keys():
        print(ref_eng_word)
        if ref_eng_word in eval_dict:
            cov, exc = evaluate_eng_word_quality(ref_dct[ref_eng_word], eval_dict[ref_eng_word])       
            eng_word_list.append(ref_eng_word)
            ref_dict_coverage.append(cov)
            exceed_pairs_generation.append(exc)  
    data = pd.DataFrame(list(zip(eng_word_list, ref_dict_coverage,exceed_pairs_generation)),
                        columns =['eng_word', 'reference_coverage','exceeded_eval_pairs'])
    return data
df = eval_dicts(ref_dct,systran_merge_mean_04_similarity_vs_pos_dict)
df.head()           

letter
plate
dinner
=======DIRECT SIM FINISHED========
['ужин_dîner', 'ужин_dîner'] ['ужин_dîner', 'ужин_dîner']
['ужин_dîner', 'ужин_dîner'] ['ужин_dîner', 'ужин_dîner']
{'ужин_dîner'} 0.5
{'ужин_dîner'} 0.8
necessary
=======DIRECT SIM FINISHED========
[] []
необходимый нужный 0.715085602657199
l1_word_sim worked
('необходимый', 'nécessaire') ('нужный', 'nécessaire')
match_found
('необходимый', 'nécessaire') ('нужный', 'indispensable')
no_match
('необходимый', 'nécessaire') ('нужный', 'requis')
no_match
('необходимый', 'nécessaire') ('нужный', 'nécessaire')
match_found
('необходимый', 'nécessaire') ('нужный', 'indispensable')
no_match
('необходимый', 'nécessaire') ('нужный', 'requis')
no_match
необходимый должный 0.5440062387986638
l1_word_sim worked
('необходимый', 'nécessaire') ('должный', 'nécessaire')
no_match
('необходимый', 'nécessaire') ('должный', 'nécessaire')
no_match
необходимый обязательный 0.6100098563462681
l1_word_sim worked
('необходимый', 'nécessaire') ('обязательный'

=======DIRECT SIM FINISHED========
['звезда_étoile', 'звезда_étoile', 'звезда_vedette', 'звезда_vedette', 'звезда_star ', 'звезда_star '] ['звезда_étoile', 'звезда_étoile', 'звезда_vedette', 'звезда_vedette', 'звезда_star', 'звезда_star']
звезда (первый ученик) звезда 0.6145184998958503
l1_word_sim worked
('звезда (первый ученик)', 'meilleur') ('звезда', 'étoile')
no_match
('звезда (первый ученик)', 'meilleur') ('звезда', 'star')
no_match
('звезда (первый ученик)', 'meilleur') ('звезда', 'vedette')
no_match
('звезда (первый ученик)', 'meilleur') ('звезда', 'étoile')
no_match
('звезда (первый ученик)', 'meilleur') ('звезда', 'star')
no_match
('звезда (первый ученик)', 'meilleur') ('звезда', 'vedette')
no_match
звезда (первый ученик) стар 0.2641785284133709
играть главную роль звезда 0.060604119962037885
играть главную роль стар -0.03183579444872556
['звезда_étoile', 'звезда_étoile', 'звезда_vedette', 'звезда_vedette', 'звезда_star ', 'звезда_star '] ['звезда_étoile', 'звезда_étoile', 'з

,eng_word,reference_coverage,exceeded_eval_pairs
0,dinner,0.5,0.800000
1,necessary,1.0,0.833333
2,table,0.5,0.777778
3,warm,1.0,0.666667
4,entire,0.0,1.000000


In [152]:
reference_dict

,English,Russian,French
0,letter,письмо,lettre
1,letter,буква,lettre
2,letter,буква,caractère
3,plate,тарелка,assiette
4,plate,тарелка,plat
5,dinner,обед,déjeuner
6,dinner,ужин,dîner
7,necessary,необходимый,nécessaire
8,inevitable,неизбежный,inévitable
9,secret,секрет,secret
